In [1]:
import numpy as np
import cv2

# 26. 모폴로지 변환

In [ ]:
# 모폴로지 변환(Morphological Transformation)
# 영상이나 이미지를 형태학적 관점에서 접근하는 기법을 의미
# 주로 영상 내 픽셀값 대체에 사용됩니다. 이를 응용해서 노이즈 제거, 요소 결합 및 분리, 강도 피크 검출 등에 이용
# 집합의 포함 관계, 이동(translation), 대칭(reflection), 여집합(complement), 차집합(difference) 등의 성질을 사용

# 기본적인 모폴로지 변환 : 팽창(dilation), 침식(erosion)

# 팽창
# 커널 영역 안에 존재하는 모든 픽셀의 값을 커널 내부의 극댓값(local maximum)으로 대체
# 어두운 영역이 줄어들고 밝은 영역이 늘어남
# 노이즈 제거 후 줄어든 크기를 복구하고자 할 때 주로 사용

# 침식
# 커널 영역 안에 존재하는 모든 픽셀의 값을 커널 내부의 극솟값(local minimum)으로 대체
# 밝은 영역이 줄어들고 어두운 영역이 늘어남
# 노이즈 제거에 주로 사용

In [26]:
src = cv2.imread('Image/zebra.jpg')

# 구조 요소 생성
# cv2.getStructuringElement(커널의 형태, 커널의 크기, 중심점)
# 커널의 형태 : 직사각형(Rect), 십자가(Cross), 타원(Ellipse)
# 커널의 크기 : 구조 요소의 크기, 커널의 크기가 너무 작다면 커널의 형태는 영향을 받지 않음
# 중심점 : 커널의 중심 위치. 필수 매개변수가 아니며, 설정하지 않을 경우 사용되는 함수에서 값이 결정(유동적)
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (9, 9))

# 모폴로지 변환(dilate:팽창, erode:침식)

# 팽창 연산 : 밝은 영역이 커짐
# cv2.dilate(원본 배열, 구조 요소, 고정점, 반복 횟수, 테두리 외삽법, 테두리 색상)
dilate = cv2.dilate(src, kernel, anchor=(-1, -1), iterations=5)

# 침식 연산 : 어두운 영역이 커짐
# cv2.erode(원본 배열, 구조 요소, 고정점, 반복 횟수, 테두리 외삽법, 테두리 색상)
erode = cv2.erode(src, kernel, anchor=(-1, -1), iterations=5)

# 고정점을 (-1, -1)로 할당할 경우, 커널의 중심부에 고정점이 위치

# 하나의 이미지로 연결
# np.concatenate((연결할 이미지 배열들), axis=방향) 0:행(세로), 1:열(가로)
# OpenCV의 함수 중, 수평 연결 함수(cv2.hconcat)와 수직 연결 함수(cv2.vconcat)도 있음
dst = np.concatenate((src, dilate, erode), axis=1)

dst = cv2.resize(dst, (0,0), fx=0.3, fy=0.3)

cv2.imshow('dst', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 27. 모폴로지 연산

In [ ]:
# 모폴로지 연산(Morphological Calculate)
# 폴로지 변환의 팽창(dilation)과 침식(erosion)을 기본 연산으로 사용해 고급 형태학을 적용하는 변환 연산
# 입력 이미지가 이진화된 이미지라면 팽창과 침식 연산으로도 우수한 결과를 얻을 수 있음
# 그레이스케일이나 다중 채널 이미지를 사용하는 경우 더 복잡한 연산을 필요로 함
# -> 이때 모폴로지 연산을 활용

# 열림(Opening)
# open = dilate(erode(src))
# 팽창 연산자와 침식 연산자의 조합, 침식->팽창
# 스펙클(speckle)이 사라지면서 발생한 객체의 크기 감소를 원래대로 복구

# 닫힘(Closing)
# close = erode(dilate(src))
# 팽창 연산자와 침식 연산자의 조합, 팽창->침식
# 객체 내부의 홀(holes)이 사라지면서 발생한 크기 증가를 원래대로 복구

# 그레이디언트(Gradient)
# gradient = dilate(src) - erode(src)
# 팽창 연산자와 침식 연산자의 조합, 각각 팽창 연산과 침식 연산을 적용하고 감산을 진행
# 입력 이미지에 객체의 가장자리가 반환
# 밝은 영역의 가장자리를 분리하며 그레이스케일 이미지가 가장 급격하게 변하는 곳에서 가장 높은 결과를 반환

# 탑햇(TopHat)
# tophat = src - open(src)
# 입력 이미지(src)와 열림(Opening)의 조합, 입력 이미지에 열림 연산을 적용한 이미지를 감산
# 밝은 영역이 분리되어 사라졌던 스펙클이나 작은 부분들이 표시됨
# 입력 이미지의 객체들이 제외되고 국소적으로 밝았던 부분들이 분리
# 열림 연산에서 사라질 요소들을 표시

# 블랙햇(BlackHat)
# blackhat = close(src) - src
# 입력 이미지(src)와 닫힘(Closing)의 조합, 닫힘 연산을 적용한 이미지에 입력 이미지를 감산
# 어두운 영역이 채워져 사라졌던 홀 등이 표시됨
# 입력 이미지의 객체들이 제외되고 국소적으로 어두웠던 홀들이 분리
# 닫힘 연산에서 사라질 요소들을 표시

# 히트미스(HitMiss)
# 단일 채널 이미지에서 활용하며, 주로 이진화 이미지에 적용
# 이미지의 전경이나 배경 픽셀의 특정 패턴을 찾는 데 사용하는 이진 형태학
# 구조 요소의 형태에 큰 영향을 받음
# 히트미스 연산의 커널은 기존 컨벌루션 커널과 다른 역할
# 내부 요소의 값은 0 또는 1의 값만 의미가 있음
# -> 모서리(Corner)를 검출하는 데 활용하기도 함
# 제한 조건 : 8-bit unsigned integers, 1-Channel

In [28]:
src = cv2.imread('Image/office.jpg', cv2.IMREAD_REDUCED_COLOR_2)

# 구조요소 생성
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (9, 9))

# cv2.morphologyEx(원본 배열, 연산 방법, 구조 요소, 고정점, 반복 횟수, 테두리 외삽법, 테두리 색상)
dst = cv2.morphologyEx(src, cv2.MORPH_OPEN, kernel, iterations=9)

cv2.imshow('dst', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 연산 방법 종류
# cv2.MORPH_DILATE	     팽창 연산
# cv2.MORPH_ERODE	     침식 연산
# cv2.MORPH_OPEN	     열림 연산
# cv2.MORPH_CLOSE	     닫힘 연산
# cv2.MORPH_GRADIENT     그레이디언트 연산
# cv2.MORPH_TOPHAT	     탑햇 연산
# cv2.MORPH_BLACKHAT     블랙햇 연산
# cv2.MORPH_HITMISS	     히트미스 연산